In [31]:
import amrlib

In [32]:
stog = amrlib.load_stog_model()
graphs = stog.parse_sents(['This is a test of the system.', 'This is a second sentence.'])
for graph in graphs:
    print(graph)

# ::snt This is a test of the system.
(t / test-01
      :ARG1 (s / system)
      :domain (t2 / this))
# ::snt This is a second sentence.
(s / sentence
      :ord (o / ordinal-entity
            :value 2)
      :domain (t / this))


In [33]:
gtos = amrlib.load_gtos_model()
sents, _ = gtos.generate(graphs)
for sent in sents:
    print(sent)

KeyboardInterrupt: 

In [4]:
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')
doc = nlp('This is a test of the SpaCy extension. The test has multiple sentences.')
graphs = doc._.to_amr()
for graph in graphs:
    print(graph)

# ::snt This is a test of the SpaCy extension.
(t / test-01
      :ARG1 (e / extend-01
            :ARG1 (p / product
                  :name (n / name
                        :op1 "SpaCy")))
      :domain (t2 / this))
# ::snt The test has multiple sentences.
(h / have-03
      :ARG0 (t / test)
      :ARG1 (s / sentence
            :quant (m / multiple)))


In [5]:
import json
import amrlib
import spacy

In [6]:
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

In [7]:
# Path to your JSONL file
file_path = "data/massive_amr.jsonl"

In [8]:
# Read the JSONL file
with open(file_path, "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

In [9]:
# Extract sentences and AMR graphs
sentences = [entry["utt"] for entry in data]
amr_graphs = [entry["raw_amr"] for entry in data]

In [10]:
# --- Test STOG: Convert Sentences to AMR ---
print("\n### STOG: Sentences to AMR ###")
parsed_graphs = stog.parse_sents(sentences[:5])  # Test on first 5 sentences
for sent, graph in zip(sentences[:5], parsed_graphs):
    print(f"\nSentence: {sent}\nAMR Graph:\n{graph}")


### STOG: Sentences to AMR ###

Sentence: what are some updates about the stock market
AMR Graph:
# ::snt what are some updates about the stock market
(u / update-02
      :ARG1 (m / market
            :mod (s / stock))
      :ARG2 (a / amr-unknown)
      :quant (s2 / some))

Sentence: definition of velocity
AMR Graph:
# ::snt definition of velocity
(d / define-01
      :ARG1 (v / velocity))

Sentence: please look up exchange between us and mexico
AMR Graph:
# ::snt please look up exchange between us and mexico
(l / look-up-05
      :polite +
      :mode imperative
      :ARG0 (y / you)
      :ARG1 (e / exchange-01
            :ARG0 (w / we)
            :ARG2 (c / country
                  :name (n / name
                        :op1 "Mexico"))))

Sentence: can you describe to me what a pineapple looks like
AMR Graph:
# ::snt can you describe to me what a pineapple looks like
(p / possible-01
      :polarity (a / amr-unknown)
      :ARG1 (d / describe-01
            :ARG0 (y / you)
  

In [11]:
# --- Test GTOS: Convert AMR to Sentences ---
print("\n### GTOS: AMR to Sentences ###")
reconstructed_sentences, _ = gtos.generate(amr_graphs[:5])  # Test on first 5 AMRs
for amr, recon_sent in zip(amr_graphs[:5], reconstructed_sentences):
    print(f"\nAMR:\n{amr}\nReconstructed Sentence: {recon_sent}")


### GTOS: AMR to Sentences ###

AMR:
(u / update-02
      :ARG2 (a / amr-unknown)
      :topic (m / market-01
            :ARG1 (s / stock))
      :mod (s2 / some))
Reconstructed Sentence: What are some stock market updates?

AMR:
(d / define-01
      :ARG1 (v / velocity)
      :ARG2 (a / amr-unknown))
Reconstructed Sentence: What is the definition of velocity?

AMR:
(l / look-up-05 :mode imperative :polite +
      :ARG0 (y / you)
      :ARG1 (e / exchange-01
            :ARG1 (c / currency
                  :mod (c3 / country :name (n / name :op1 "us")))
            :ARG3 (c2 / currency
                  :mod (c4 / country :name (n2 / name :op1 "mexico")))))
Reconstructed Sentence: Please look up exchange rates between US and Mexican currency.

AMR:
(d / describe-01 :mode imperative :polite +
      :ARG0 (y / you)
      :ARG1 (t / thing
            :ARG1-of (l / look-02
                  :ARG0 (f / food-dish :name (n / name :op1 "pineapple")))))
Reconstructed Sentence: Please describ

In [12]:
# --- Test SpaCy + AMR ---
print("\n### SpaCy AMR Extension ###")
for sent in sentences[:3]:  # Test on first 3 sentences
    doc = nlp(sent)
    doc_graphs = doc._.to_amr()
    for graph in doc_graphs:
        print(f"\nSentence: {sent}\nSpaCy AMR Graph:\n{graph}")


### SpaCy AMR Extension ###

Sentence: what are some updates about the stock market
SpaCy AMR Graph:
# ::snt what are some updates about the stock market
(u / update-02
      :ARG1 (m / market
            :mod (s / stock))
      :ARG2 (a / amr-unknown)
      :quant (s2 / some))

Sentence: definition of velocity
SpaCy AMR Graph:
# ::snt definition of velocity
(d / define-01
      :ARG1 (v / velocity))

Sentence: please look up exchange between us and mexico
SpaCy AMR Graph:
# ::snt please look up exchange between us and mexico
(l / look-up-05
      :polite +
      :mode imperative
      :ARG0 (y / you)
      :ARG1 (e / exchange-01
            :ARG0 (w / we)
            :ARG2 (c / country
                  :name (n / name
                        :op1 "Mexico"))))


# Let's try to translate the dataset to Irish

In [41]:
import json
from transformers import MarianMTModel, MarianTokenizer

In [42]:
# ----------------------------
# 1. Load Translation Model (English → Irish)
# ----------------------------

In [43]:
model_name = "Helsinki-NLP/opus-mt-en-ga"  # English to Irish
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [44]:
# ----------------------------
# 2. Load JSONL Data
# ----------------------------

In [45]:
input_file = "data/massive_amr.jsonl"
output_file = "data/massive_amr_irish.jsonl"

In [46]:
with open(input_file, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [47]:
# ----------------------------
# 3. Translate Function
# ----------------------------

In [48]:
def translate_text(text, tokenizer, model):
    """Translates English text to Irish using MarianMT."""
    if not text.strip():
        return text  # Skip empty strings

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    return translated_text

In [49]:
# ----------------------------
# 4. Translate Sentences
# ----------------------------

In [50]:
for entry in data:
    entry["utt"] = translate_text(entry["utt"], tokenizer, model)
    entry["annot_utt"] = translate_text(entry["annot_utt"], tokenizer, model)

In [51]:
# ----------------------------
# 5. Save Translated Data
# ----------------------------

In [52]:
with open(output_file, "w", encoding="utf-8") as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

In [53]:
print("✅ Translation complete! Saved as 'massive_amr_irish.jsonl'")

✅ Translation complete! Saved as 'massive_amr_irish.jsonl'


# Let's try to train the AMR Parser on Welsh - T5-base did not worked

In [1]:
import amrlib
print(amrlib.__version__)  # Should print the version number
print(hasattr(amrlib, 'parse_string')) # Should print True

0.8.0
False


In [1]:
import json
import os
import random
import numpy as np
import torch
import gc
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

In [2]:
# Free GPU memory before training
torch.cuda.empty_cache()
gc.collect()

20

In [3]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

torch.cuda.empty_cache()

In [4]:
# Load Data
file_path = "data/massive_amr_welsh.jsonl"

In [5]:
def load_amr_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line) for line in file]
    return [entry for entry in data if entry.get("raw_amr") and entry.get("utt")]

In [6]:
data = load_amr_data(file_path)
random.shuffle(data)

In [7]:
# Split Data
sentences = [entry["utt"] for entry in data]
amrs = [entry["raw_amr"] for entry in data]
train_sents, test_sents, train_amrs, test_amrs = train_test_split(sentences, amrs, test_size=0.1, random_state=42)
train_sents, val_sents, train_amrs, val_amrs = train_test_split(train_sents, train_amrs, test_size=0.1, random_state=42)

In [8]:
# Convert to Hugging Face Dataset format
def create_dataset(sentences, amrs):
    return Dataset.from_dict({"sentence": sentences, "amr": amrs})

datasets = DatasetDict({
    "train": create_dataset(train_sents, train_amrs),
    "validation": create_dataset(val_sents, val_amrs),
    "test": create_dataset(test_sents, test_amrs),
})

In [9]:
# Load T5 Model
model_name = "t5-base"  # Use "t5-small" if GPU still struggles
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
# Enable Gradient Checkpointing (Reduces Memory Usage)
model.gradient_checkpointing_enable()


In [11]:
# Tokenization Function
def preprocess_function(examples):
    inputs = ["parse: " + ex for ex in examples["sentence"]]
    targets = examples["amr"]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=1024, truncation=True, padding="max_length")
    
    # Replace padding token id's in labels with -100 so they are ignored by the loss function
    labels["input_ids"] = [
        [l if l != tokenizer.pad_token_id else -100 for l in label]
        for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# Apply tokenization
tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=["sentence", "amr"])

Map:   0%|          | 0/1364 [00:00<?, ? examples/s]

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
# Optimized Training Arguments for RTX 3070 Ti
training_args = TrainingArguments(
    output_dir="./amr_t5_model_optimized",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,  # Lower LR for stability
    lr_scheduler_type="linear",
    warmup_steps=500,
    per_device_train_batch_size=2,  # Reduce batch size
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # Adjusted to reduce memory usage
    num_train_epochs=10,  # Reduced epochs to avoid overfitting
    weight_decay=0.03,  # Regularization
    save_total_limit=2,
    logging_steps=50,
    logging_dir="./logs",
    report_to="none",
    fp16=True,  # Mixed Precision Training (Lowers Memory Usage)
)

c:\Users\stoic\miniconda3\envs\llms_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\stoic\AppData\Local\Temp\ipykernel_18480\4006242607.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
import gc
torch.cuda.empty_cache()
gc.collect()

187

In [17]:
print("🚀 Starting Optimized T5 Training for AMR Parsing...")
trainer.train()

🚀 Starting Optimized T5 Training for AMR Parsing...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [18]:
print("Dataset Columns:", datasets["test"].column_names)

Dataset Columns: ['sentence', 'amr']


In [19]:
print("Dataset Columns (after tokenization):", tokenized_datasets["test"].column_names)

Dataset Columns (after tokenization): ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']


In [20]:
# ----------------------------
# 9. Save Fine-Tuned Model
# ----------------------------
model.save_pretrained("./fine_tuned_amr_t5_2")
tokenizer.save_pretrained("./fine_tuned_amr_t5_2")

('./fine_tuned_amr_t5_2\\tokenizer_config.json',
 './fine_tuned_amr_t5_2\\special_tokens_map.json',
 './fine_tuned_amr_t5_2\\spiece.model',
 './fine_tuned_amr_t5_2\\added_tokens.json')

# Calculate Smatch Score - T5-base did not worked

In [18]:
import json
import torch
import smatch
import re
from collections import defaultdict
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset

In [19]:
# -----------------------------------
# ⿡ Load Fine-Tuned T5 Model & Tokenizer
# -----------------------------------
model_path = "./fine_tuned_amr_t5_2"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)


In [20]:
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [21]:
# -----------------------------------
# ⿢ Load Test Data
# -----------------------------------
file_path = "data/massive_amr_welsh.jsonl"


In [22]:
def load_amr_data(file_path):
    """Loads JSONL AMR dataset and removes missing entries."""
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line) for line in file]
    return [entry for entry in data if entry.get("raw_amr") and entry.get("utt")]


In [23]:
# Load test data
data = load_amr_data(file_path)
sentences = [entry["utt"] for entry in data]
gold_amrs = [entry["raw_amr"] for entry in data]


In [24]:
# Convert to Hugging Face Dataset format
test_dataset = Dataset.from_dict({"sentence": sentences, "amr": gold_amrs})


In [25]:
# -----------------------------------
# ⿣ Improved AMR Cleaning Function (Fix Duplicate Names)
# -----------------------------------
def fix_duplicate_nodes(amr_text):
    """Renames duplicate variables dynamically."""
    if not amr_text.strip():
        return "INVALID_AMR"

    used_vars = defaultdict(int)  # Tracks occurrences of each variable
    renamed_vars = {}

    def rename_variable(match):
        var_name = match.group(1)
        if var_name in used_vars:
            new_var_name = f"{var_name}_{used_vars[var_name]}"
            used_vars[var_name] += 1
            renamed_vars[var_name] = new_var_name
            return f"({new_var_name} /"
        else:
            used_vars[var_name] = 1
            return f"({var_name} /"

    # Ensure variables like (t2 / thing) are renamed uniquely
    amr_text = re.sub(r"\((\w+)\s+\/", rename_variable, amr_text)

    # Replace occurrences of renamed variables in AMR text
    for old_var, new_var in renamed_vars.items():
        amr_text = amr_text.replace(f" {old_var} ", f" {new_var} ")

    return amr_text


In [26]:
# -----------------------------------
# ⿤ Generate AMR Predictions (Fixed)
# -----------------------------------
def generate_amr_predictions(model, tokenizer, test_dataset, max_samples=100):
    """Generates AMR graphs from input sentences using the fine-tuned model."""
    predictions = []

    for i, example in enumerate(test_dataset):
        if i >= max_samples:  # Limit number of samples for efficiency
            break

        input_text = "parse: " + example["sentence"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=512)
        
        predicted_amr = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predicted_amr = fix_duplicate_nodes(predicted_amr)  # ✅ Ensure AMR is valid

        gold_amr = fix_duplicate_nodes(example["amr"])  # ✅ Fix duplicates in gold AMRs too

        predictions.append((gold_amr, predicted_amr))

    return predictions


In [27]:
# Generate predictions
predictions = generate_amr_predictions(model, tokenizer, test_dataset)


In [28]:
# -----------------------------------
# ⿥ Compute Smatch Score (with Fixes)
# -----------------------------------
def is_valid_amr(amr):
    """Checks if AMR is valid by ensuring it has at least one '/' symbol."""
    return amr.count('/') > 0


In [29]:
def compute_smatch(gold_amrs, predicted_amrs):
    """Computes the Smatch score while handling errors."""
    total_precision, total_recall, total_f1 = 0, 0, 0
    valid_samples = 0

    for i, (gold, pred) in enumerate(zip(gold_amrs, predicted_amrs)):
        try:
            # ✅ Ensure AMRs are valid
            if not is_valid_amr(pred) or not is_valid_amr(gold):
                print(f"⚠ Skipping sample {i} due to invalid AMR")
                continue

            precision, recall, f_score = smatch.get_amr_match(str(gold), str(pred))
            total_precision += precision
            total_recall += recall
            total_f1 += f_score
            valid_samples += 1
        except Exception as e:
            print(f"⚠ Error in Smatch calculation for sample {i}: {e}")
            print(f"Gold AMR: {gold}")
            print(f"Predicted AMR: {pred}")
            print("-" * 50)

    if valid_samples == 0:
        return 0, 0, 0  # Avoid division by zero

    return total_precision / valid_samples, total_recall / valid_samples, total_f1 / valid_samples


In [30]:
# Extract predicted AMRs for Smatch evaluation
gold_amrs = [gold for gold, _ in predictions if gold is not None]
predicted_amrs = [pred for _, pred in predictions if pred is not None]


In [31]:
# Compute Smatch Score
precision, recall, f1 = compute_smatch(gold_amrs, predicted_amrs)


⚠ Skipping sample 0 due to invalid AMR
⚠ Skipping sample 1 due to invalid AMR
⚠ Skipping sample 2 due to invalid AMR
⚠ Skipping sample 3 due to invalid AMR
⚠ Skipping sample 4 due to invalid AMR
⚠ Skipping sample 5 due to invalid AMR
⚠ Skipping sample 6 due to invalid AMR
⚠ Skipping sample 7 due to invalid AMR
⚠ Skipping sample 8 due to invalid AMR
⚠ Skipping sample 9 due to invalid AMR
⚠ Skipping sample 10 due to invalid AMR
⚠ Skipping sample 11 due to invalid AMR
⚠ Skipping sample 12 due to invalid AMR
⚠ Skipping sample 13 due to invalid AMR
⚠ Skipping sample 14 due to invalid AMR
⚠ Skipping sample 15 due to invalid AMR
⚠ Skipping sample 16 due to invalid AMR
⚠ Skipping sample 17 due to invalid AMR
⚠ Skipping sample 18 due to invalid AMR
⚠ Skipping sample 19 due to invalid AMR
⚠ Skipping sample 20 due to invalid AMR
⚠ Skipping sample 21 due to invalid AMR
⚠ Skipping sample 22 due to invalid AMR
⚠ Skipping sample 23 due to invalid AMR
⚠ Skipping sample 24 due to invalid AMR
⚠ Skipping

In [32]:
# -----------------------------------
# ⿦ Display Results
# -----------------------------------
print(f"🔥 Smatch Score - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


🔥 Smatch Score - Precision: 0.0000, Recall: 0.0000, F1: 0.0000


# Check Dataset for Corruption

In [90]:
import json

In [91]:
# Load dataset
file_path = "data/massive_amr_welsh.jsonl"
valid_entries = []

In [92]:
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            entry = json.loads(line)
            if "utt" in entry and "raw_amr" in entry and isinstance(entry["utt"], str) and isinstance(entry["raw_amr"], str):
                valid_entries.append(entry)
            else:
                print(f"❌ Corrupt entry found and skipped: {entry}")
        except json.JSONDecodeError:
            print(f"❌ JSON Decode Error in line: {line}")

In [93]:
print(f"✅ Valid samples: {len(valid_entries)}")

✅ Valid samples: 1685


In [94]:
# Save cleaned dataset
with open("data/cleaned_amr_welsh.jsonl", "w", encoding="utf-8") as f:
    for entry in valid_entries:
        f.write(json.dumps(entry) + "\n")

print("✅ Cleaned dataset saved as 'data/cleaned_amr_welsh.jsonl'")


✅ Cleaned dataset saved as 'data/cleaned_amr_welsh.jsonl'


# Good Smatch Score for Welsh - T5-small

In [34]:
import json
import torch
import smatch
import re
from collections import defaultdict
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset

# -----------------------------------
# ⿡ Load Fine-Tuned T5 Model & Tokenizer
# -----------------------------------
model_path = "./fine_tuned_amr_t5"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# -----------------------------------
# ⿢ Load Test Data
# -----------------------------------
file_path = "data/massive_amr_welsh.jsonl"

def load_amr_data(file_path):
    """Loads JSONL AMR dataset and removes missing entries."""
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line) for line in file]
    return [entry for entry in data if entry.get("raw_amr") and entry.get("utt")]

# Load test data
data = load_amr_data(file_path)
sentences = [entry["utt"] for entry in data]
gold_amrs = [entry["raw_amr"] for entry in data]

# Convert to Hugging Face Dataset format
test_dataset = Dataset.from_dict({"sentence": sentences, "amr": gold_amrs})

# -----------------------------------
# ⿣ Improved AMR Cleaning Function (Fix Duplicate Names)
# -----------------------------------
def fix_duplicate_nodes(amr_text):
    """Renames duplicate variables dynamically."""
    if not amr_text.strip():
        return "INVALID_AMR"

    used_vars = defaultdict(int)  # Tracks occurrences of each variable
    renamed_vars = {}

    def rename_variable(match):
        var_name = match.group(1)
        if var_name in used_vars:
            new_var_name = f"{var_name}_{used_vars[var_name]}"
            used_vars[var_name] += 1
            renamed_vars[var_name] = new_var_name
            return f"({new_var_name} /"
        else:
            used_vars[var_name] = 1
            return f"({var_name} /"

    # Ensure variables like (t2 / thing) are renamed uniquely
    amr_text = re.sub(r"\((\w+)\s+\/", rename_variable, amr_text)

    # Replace occurrences of renamed variables in AMR text
    for old_var, new_var in renamed_vars.items():
        amr_text = amr_text.replace(f" {old_var} ", f" {new_var} ")

    return amr_text

# -----------------------------------
# ⿤ Generate AMR Predictions (Fixed)
# -----------------------------------
def generate_amr_predictions(model, tokenizer, test_dataset, max_samples=100):
    """Generates AMR graphs from input sentences using the fine-tuned model."""
    predictions = []

    for i, example in enumerate(test_dataset):
        if i >= max_samples:  # Limit number of samples for efficiency
            break

        input_text = "parse: " + example["sentence"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=512)
        
        predicted_amr = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predicted_amr = fix_duplicate_nodes(predicted_amr)  # ✅ Ensure AMR is valid

        gold_amr = fix_duplicate_nodes(example["amr"])  # ✅ Fix duplicates in gold AMRs too

        predictions.append((gold_amr, predicted_amr))

    return predictions

# Generate predictions
predictions = generate_amr_predictions(model, tokenizer, test_dataset)

# -----------------------------------
# ⿥ Compute Smatch Score (with Fixes)
# -----------------------------------
def is_valid_amr(amr):
    """Checks if AMR is valid by ensuring it has at least one '/' symbol."""
    return amr.count('/') > 0

def compute_smatch(gold_amrs, predicted_amrs):
    """Computes the Smatch score while handling errors."""
    total_precision, total_recall, total_f1 = 0, 0, 0
    valid_samples = 0

    for i, (gold, pred) in enumerate(zip(gold_amrs, predicted_amrs)):
        try:
            # ✅ Ensure AMRs are valid
            if not is_valid_amr(pred) or not is_valid_amr(gold):
                print(f"⚠ Skipping sample {i} due to invalid AMR")
                continue

            precision, recall, f_score = smatch.get_amr_match(str(gold), str(pred))
            total_precision += precision
            total_recall += recall
            total_f1 += f_score
            valid_samples += 1
        except Exception as e:
            print(f"⚠ Error in Smatch calculation for sample {i}: {e}")
            print(f"Gold AMR: {gold}")
            print(f"Predicted AMR: {pred}")
            print("-" * 50)

    if valid_samples == 0:
        return 0, 0, 0  # Avoid division by zero

    return total_precision / valid_samples, total_recall / valid_samples, total_f1 / valid_samples

# Extract predicted AMRs for Smatch evaluation
gold_amrs = [gold for gold, _ in predictions if gold is not None]
predicted_amrs = [pred for _, pred in predictions if pred is not None]

# Compute Smatch Score
precision, recall, f1 = compute_smatch(gold_amrs, predicted_amrs)

# -----------------------------------
# ⿦ Display Results
# -----------------------------------
print(f"🔥 Smatch Score - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

⚠ Error in Smatch calculation for sample 7: 'NoneType' object has no attribute 'rename_node'
Gold AMR: (r / rate-01
      :ARG1 (e / exchange-01
            :ARG1 (c / currency :name (n / name :op1 "u." :op2 "s." :op3 "d."))
            :ARG3 (c2 / currency :name (n2 / name :op1 "cdn")))
      :ARG2 (a / amr-unknown))
Predicted AMR: (r / rate-01 :ARG1 (e / exchange-01 :ARG1 (c / currency :name (n / name :op1 "u." :op2 "s." :op3 "d.")) :ARG3 (c2 / currency :name (n2 / name :op1 "d."))) :ARG3 (c2_1 / currency :name (n2_1 / name :op1 "euro"))))
--------------------------------------------------
⚠ Error in Smatch calculation for sample 12: not enough values to unpack (expected 2, got 1)
Gold AMR: (t / tell-01 :mode imperative
      :ARG0 (y / you)
      :ARG1 (t2 / thing
            :mod (a2 / all)
            :topic (h / hurricane-01))
      :ARG2 (i / i))
Predicted AMR: (t / tell-01 :mode imperative :ARG0 (y / you) :ARG1 (t2 / thing :ARG2-of (p / person :name (n / name :op1 "abraham" :op

Unmatched parenthesis at position 211 in processing (r / rate-01 :ARG1 (e / exchange-01 :ARG1 (c / currency :name (n / name :op1 "u." :op2 "s." :op3 "d.")) :ARG3 (c2 / currency :name (n2 / name :op1 "d."))) :ARG3 (c2_1 / currency :name (n2_1 / name :op1 "euro"))))
Error in parsing amr 2: (t / tell-01 :mode imperative :ARG0 (y / you) :ARG1 (t2 / thing :ARG2-of (p / person :name (n / name :op1 "abraham" :op2 "s." :op3 "s." :op3 "s." :op3 "m.")))) :ARG2 (i / i))
Please check if the AMR is ill-formatted. Ignoring remaining AMRs
Error message: list index out of range
Error in parsing amr 2: (r / rate-01 :ARG1 (e / exchange-01 :ARG1 (c / currency :name (n / name :op1 "euro")) :ARG3 (c2 / currency :name (n2 / name :op1 "euro"))) :ARG3 (c2_1 / currency :name (n2_1 / name :op1 "euro"))) :ARG2 (a / amr-unknown))
Please check if the AMR is ill-formatted. Ignoring remaining AMRs
Error message: list index out of range
Error in parsing amr 2: (t / tell-01 :mode imperative :polite + :ARG0 (y / you) :

# Smatch score for T5-base - did not worked

In [36]:
import json
import torch
import smatch
import re
from collections import defaultdict
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset

# -----------------------------------
# ⿡ Load Fine-Tuned T5 Model & Tokenizer
# -----------------------------------
model_path = "./fine_tuned_amr_t5_2"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# -----------------------------------
# ⿢ Load Test Data
# -----------------------------------
file_path = "data/massive_amr_welsh.jsonl"

def load_amr_data(file_path):
    """Loads JSONL AMR dataset and removes missing entries."""
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line) for line in file]
    return [entry for entry in data if entry.get("raw_amr") and entry.get("utt")]

# Load test data
data = load_amr_data(file_path)
sentences = [entry["utt"] for entry in data]
gold_amrs = [entry["raw_amr"] for entry in data]

# Convert to Hugging Face Dataset format
test_dataset = Dataset.from_dict({"sentence": sentences, "amr": gold_amrs})

# -----------------------------------
# ⿣ Improved AMR Cleaning Function (Fix Duplicate Names)
# -----------------------------------
def fix_duplicate_nodes(amr_text):
    """Renames duplicate variables dynamically."""
    if not amr_text.strip():
        return "INVALID_AMR"

    used_vars = defaultdict(int)  # Tracks occurrences of each variable
    renamed_vars = {}

    def rename_variable(match):
        var_name = match.group(1)
        if var_name in used_vars:
            new_var_name = f"{var_name}_{used_vars[var_name]}"
            used_vars[var_name] += 1
            renamed_vars[var_name] = new_var_name
            return f"({new_var_name} /"
        else:
            used_vars[var_name] = 1
            return f"({var_name} /"

    # Ensure variables like (t2 / thing) are renamed uniquely
    amr_text = re.sub(r"\((\w+)\s+\/", rename_variable, amr_text)

    # Replace occurrences of renamed variables in AMR text
    for old_var, new_var in renamed_vars.items():
        amr_text = amr_text.replace(f" {old_var} ", f" {new_var} ")

    return amr_text

# -----------------------------------
# ⿤ Generate AMR Predictions (Fixed)
# -----------------------------------
def generate_amr_predictions(model, tokenizer, test_dataset, max_samples=100):
    """Generates AMR graphs from input sentences using the fine-tuned model."""
    predictions = []

    for i, example in enumerate(test_dataset):
        if i >= max_samples:  # Limit number of samples for efficiency
            break

        input_text = "parse: " + example["sentence"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=512)
        
        predicted_amr = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predicted_amr = fix_duplicate_nodes(predicted_amr)  # ✅ Ensure AMR is valid

        gold_amr = fix_duplicate_nodes(example["amr"])  # ✅ Fix duplicates in gold AMRs too

        predictions.append((gold_amr, predicted_amr))

    return predictions

# Generate predictions
predictions = generate_amr_predictions(model, tokenizer, test_dataset)

# -----------------------------------
# ⿥ Compute Smatch Score (with Fixes)
# -----------------------------------
def is_valid_amr(amr):
    """Checks if AMR is valid by ensuring it has at least one '/' symbol."""
    return amr.count('/') > 0

def compute_smatch(gold_amrs, predicted_amrs):
    """Computes the Smatch score while handling errors."""
    total_precision, total_recall, total_f1 = 0, 0, 0
    valid_samples = 0

    for i, (gold, pred) in enumerate(zip(gold_amrs, predicted_amrs)):
        try:
            # ✅ Ensure AMRs are valid
            if not is_valid_amr(pred) or not is_valid_amr(gold):
                print(f"⚠ Skipping sample {i} due to invalid AMR")
                continue

            precision, recall, f_score = smatch.get_amr_match(str(gold), str(pred))
            total_precision += precision
            total_recall += recall
            total_f1 += f_score
            valid_samples += 1
        except Exception as e:
            print(f"⚠ Error in Smatch calculation for sample {i}: {e}")
            print(f"Gold AMR: {gold}")
            print(f"Predicted AMR: {pred}")
            print("-" * 50)

    if valid_samples == 0:
        return 0, 0, 0  # Avoid division by zero

    return total_precision / valid_samples, total_recall / valid_samples, total_f1 / valid_samples

# Extract predicted AMRs for Smatch evaluation
gold_amrs = [gold for gold, _ in predictions if gold is not None]
predicted_amrs = [pred for _, pred in predictions if pred is not None]

# Compute Smatch Score
precision, recall, f1 = compute_smatch(gold_amrs, predicted_amrs)

# -----------------------------------
# ⿦ Display Results
# -----------------------------------
print(f"🔥 Smatch Score - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

KeyboardInterrupt: 